In [1]:
# Emissions Calculation by Vehicle Type


In [132]:
import pandas as pd

In [133]:
def grams_to_tons(value):
	""" Convert grams to tons."""

	value = value/453.592
	value = value/2000

	return value

In [134]:
def calculate_tons(df):
    df.rename(columns={'geog_name':'county', 'avgspeedbinId': 'avgSpeedBinID', 'roadtypeId': 'roadTypeID', 'hourId': 'hourID'},
              inplace=True)

    # Calculate total VMT by vehicle group
    df['light'] = df['sov_vmt']+df['hov2_vmt']+df['hov3_vmt']
    df['medium'] = df['medium_truck_vmt']
    df['heavy'] = df['heavy_truck_vmt']
    # What about buses??
    df.drop(['sov_vmt','hov2_vmt','hov3_vmt','medium_truck_vmt','heavy_truck_vmt','bus_vmt'], inplace=True, axis=1)

    # Melt to pivot vmt by vehicle type columns as rows
    df = pd.melt(df, id_vars=['avgSpeedBinID','roadTypeID','hourID','county'], var_name='veh_type', value_name='vmt')

    newdf = pd.merge(df, df_rates, on=['avgSpeedBinID','roadTypeID','hourID','county','veh_type'], how='left', left_index=False)
    # Calculate total grams of emission 
    newdf['grams_tot'] = newdf['grams_per_mile']*newdf['vmt']
    newdf['tons_tot'] = grams_to_tons(newdf['grams_tot'])
    
    return newdf

In [135]:
# Load rate files
df_rates = pd.DataFrame()
for county in ['King','Kitsap','Pierce','Snohomish']:
#     df1 = pd.read_csv(r'Y:\Air Quality\2018 Update MOVES\RawOutput\2040ghg\\' + county + '_2040_light.csv')
    df1 = pd.read_csv(r'Y:\Air Quality\Vision2050\2050\output\by_veh_type\\'+county.lower()+'_2050_grouped_light.csv')
    df1['veh_type'] = 'light'
    df1['county'] = county
#     df2 = pd.read_csv(r'Y:\Air Quality\2018 Update MOVES\RawOutput\2040ghg\\' + county + '_2040_med.csv')
    df2 = pd.read_csv(r'Y:\Air Quality\Vision2050\2050\output\by_veh_type\\'+county.lower()+'_2050_grouped_medium.csv')
    df2['veh_type'] = 'medium'
    df2['county'] = county
#     df3 = pd.read_csv(r'Y:\Air Quality\2018 Update MOVES\RawOutput\2040ghg\\' + county + '_2040_heavy.csv')
    df3 = pd.read_csv(r'Y:\Air Quality\Vision2050\2050\output\by_veh_type\\'+county.lower()+'_2050_grouped_heavy.csv')
    df3['veh_type'] = 'heavy'
    df3['county'] = county
    
    df_rates = df_rates.append(df1)
    df_rates = df_rates.append(df2)
    df_rates = df_rates.append(df3)

df_rates.rename(columns={'sum(ratePerDistance)': 'grams_per_mile'}, inplace=True)
# df_rates.rename(columns={'avgspeedbinId': 'avgSpeedBinID', 'roadtypeId': 'roadTypeID', 'hourId': 'hourID',
#                         'gramsPerMile': 'grams_per_mile'}, inplace=True)

# Interzonal Trips

In [136]:
df_inter = pd.read_csv(r'L:\vision2050\soundcast\integrated\final_runs\tod\tod_run_8.run_2018_10_29_15_01\2050\outputs\emissions\interzonal_vmt_grouped.csv')
df_inter = calculate_tons(df_inter)

# Intrazonal Trips

In [137]:
df_intra = pd.read_csv(r'L:\vision2050\soundcast\integrated\final_runs\tod\tod_run_8.run_2018_10_29_15_01\2050\outputs\emissions\intrazonal_vmt_grouped.csv')
df_intra.rename(columns={'vehicle_type':'veh_type', 'VMT': 'vmt', 'hourId': 'hourID', 'geog_name': 'county'},inplace=True)
df_intra.drop('tod', axis=1, inplace=True)

df_intra_light = df_intra[df_intra['veh_type'].isin(['sov','hov2','hov3'])]
df_intra_light = df_intra_light.groupby(['county','hourID']).sum()[['vmt']].reset_index()
df_intra_light['veh_type'] = 'light'

df_intra_medium = df_intra[df_intra['veh_type'] == 'mediumtruck']
df_intra_medium['veh_type'] = 'medium'
df_intra_heavy = df_intra[df_intra['veh_type'] == 'heavytruck']
df_intra_heavy['veh_type'] = 'heavy'

df_intra = df_intra_light.append(df_intra_medium)
df_intra = df_intra.append(df_intra_heavy)

# For intrazonals, assume standard speed bin and roadway type for all intrazonal trips
speedbin = 4
roadtype = 5

iz_rates = df_rates[(df_rates['avgSpeedBinID'] == speedbin) &
                    (df_rates['roadTypeID'] == roadtype)]

df_intra = pd.merge(df_intra, iz_rates, on=['hourID','county','veh_type'], how='left', left_index=False)
# Calculate total grams of emission 
df_intra['grams_tot'] = df_intra['grams_per_mile']*df_intra['vmt']
df_intra['tons_tot'] = grams_to_tons(df_intra['grams_tot'])

In [138]:
df_intra[df_intra['pollutantID'] == 98]['tons_tot'].sum()

153.3029325439923

# Starts

In [129]:
base_year = '2014'
model_year = '2050'

start_rates_df = pd.DataFrame()
for county in ['King','Kitsap','Pierce','Snohomish']:
#     for veh_type in ['light','med','heavy']:
    for veh_type in ['light','medium','heavy']:
        df = pd.read_csv(r'Y:\Air Quality\Vision2050\2050\output\by_veh_type\\'+county.lower()+'_2050_'+veh_type+'_starts.csv')
#         df = pd.read_csv(r'Y:\Air Quality\2018 Update MOVES\RawOutput\2040ghg\starts_'+county.lower()+'_2040_'+veh_type+'.csv')
    # For GHG analysis, all starts are in April
#         Sum total emissions across processID 
        df = df.groupby('pollutantID').sum()[['ratePerVehicle']].reset_index()
        if veh_type == 'med':
            veh_type = 'medium'
        df['veh_type'] = veh_type
        df['county'] = county
        start_rates_df = start_rates_df.append(df)

In [130]:
# Estimate vehicle population for AQ purposes (not using Soundcast estimates)
# Ref: X:\Trans\AIRQUAL\T2040 2018 Update\EmissionCalcs\Start Emissions\Estimate Vehicle Population_updatedfor2018.xlsx
# Pivoting from 2014 Vehicle population data and 2040 projections -> updated to 2050 using this process

# For GHG Analysis see:
# X:\Trans\AIRQUAL\T2040 2018 Update\EmissionCalcs\Start Emissions\Starts_2040_GHG.xlsx
veh_totals = {'2014': 3176086, '2040': 3982578.1, '2050': 4437371}

# Base year distribution of vehicle ownership by county
vehs_by_county = {
    'King': 1625471,
    'Kitsap': 231231,
    'Pierce': 675660,
    'Snohomish': 643724
}

# Base year distribution of vehicles by type (2014)
# Note that the totals by county above (for 2014) 
# include buses, so the totals below do not match up
# The difference is buses
vehs_by_type = {
    'King': {
        'light': 1433538,
        'medium': 174597,
        'heavy': 10574
    },
    'Kitsap': {
        'light': 199872,
        'medium': 28830,
        'heavy': 1342
    },
    'Pierce': {
        'light': 585681,
        'medium': 82607,
        'heavy': 4305
    },
    'Snohomish': {
        'light': 559557,
        'medium': 77685,
        'heavy': 3884
    },
}

base_county_veh = pd.DataFrame.from_dict(vehs_by_type, orient='index')
base_county_veh

# Scale county vehicles by total change
veh_scale = 1+(veh_totals[model_year]-veh_totals[base_year])/veh_totals[base_year]

# # Apply scale factor to the base vehicle sum by county
scen_county_veh = base_county_veh*veh_scale

vehicles_df = pd.DataFrame(scen_county_veh.unstack()).reset_index()
vehicles_df.columns = ['veh_type','county','vehicles']

# Join with rates to calculate total emissions
start_emissions_df = pd.merge(vehicles_df, start_rates_df, on=['veh_type','county'])

start_emissions_df['start_grams'] = start_emissions_df['vehicles']*start_emissions_df['ratePerVehicle'] 
start_emissions_df['start_tons'] = grams_to_tons(start_emissions_df['start_grams'])

start_emissions_df = start_emissions_df.groupby(['pollutantID','veh_type','county']).sum().reset_index()

start_emissions_df[start_emissions_df['pollutantID'] == 98]

,pollutantID,veh_type,county,vehicles,ratePerVehicle,start_grams,start_tons
102,98,heavy,King,10574,490.873480,5.190496e+06,5.721547
103,98,heavy,Kitsap,1342,490.244460,6.579081e+05,0.725220
104,98,heavy,Pierce,4305,446.431060,1.921886e+06,2.118518
105,98,heavy,Snohomish,3884,491.372980,1.908493e+06,2.103755
106,98,light,King,1433538,545.660171,7.822246e+08,862.255717
107,98,light,Kitsap,199872,537.222659,1.073758e+08,118.361619
108,98,light,Pierce,585681,505.664985,2.961584e+08,326.458992
109,98,light,Snohomish,559557,547.163305,3.061691e+08,337.493890
110,98,medium,King,174597,701.475910,1.224756e+08,135.006338
111,98,medium,Kitsap,28830,701.211920,2.021594e+07,22.284277
